# Sudoku-15-Infer-Python : Resolution Probabiliste avec Pyro et PyMC

**Navigation** : [<< Choco](Sudoku-11-Choco-Python.ipynb) | [Index](README.md) | [Neural Network >>](Sudoku-16-NeuralNetwork-Python.ipynb)

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. **Comprendre** les capacites et limites de Pyro/NumPyro et PyMC pour les problemes discrets
2. **Implementer** un modele probabiliste avec distributions Dirichlet et contraintes douces
3. **Utiliser** l'enumeration discretes et les potentials pour les contraintes
4. **Comparer** differentes strategies d'inference (SVI, MCMC, enumeration)

**Duree estimee** : ~45 min | **Prerequis** : Sudoku-0 Environment, probabilites bayesiennes

---

## Introduction : Programmation Probabiliste et Sudoku

Ce notebook explore comment utiliser **Pyro/NumPyro** et **PyMC** pour resoudre des Sudokus. Contrairement a Infer.NET (C#) qui utilise Expectation Propagation avec contraintes dures, ces frameworks Python utilisent principalement :

| Framework | Algorithmes | Contraintes | Variables discretes |
|-----------|-------------|-------------|---------------------|
| **Infer.NET** | Expectation Propagation | Dures (`ConstrainFalse`) | Natives |
| **NumPyro** | SVI, MCMC (NUTS) | Douces (penalites) | Enumeration requise |
| **PyMC** | MCMC (NUTS, Metropolis) | Douces (potentials) | Natives mais lent |

### Strategie adoptee

Puisque les contraintes de Sudoku sont discretes et difficiles a gerer en inference variationnelle, nous utilisons une approche **hybride** :

1. **Modele probabiliste** : Distributions Dirichlet pour les probabilites des cellules
2. **Inference** : SVI ou MCMC pour apprendre les distributions
3. **Resolution** : Fixation iterative des cellules les plus certaines + propagation de contraintes

## 1. Installation et Imports

In [ ]:
# Installation des dependances
import subprocess
import sys

def install_if_needed(package, import_name=None):
    import_name = import_name or package
    try:
        __import__(import_name)
        print(f"{package} deja installe")
    except ImportError:
        print(f"Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])

# Pyro/NumPyro
install_if_needed("numpyro")
install_if_needed("jax", "jax")

# PyMC (optionnel - peut etre lourd)
# install_if_needed("pymc", "pymc")

import numpy as np
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import SVI, Trace_ELBO, Predictive, MCMC, NUTS
from numpyro.optim import Adam
from numpyro import handlers
from jax import random, jit, vmap
import time
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass

# Desactiver les warnings JAX pour plus de lisibilite
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# Configuration
print(f"JAX version: {jax.__version__}")
print(f"NumPyro version: {numpyro.__version__}")
print(f"Backend: {jax.devices()}")

## 2. Utilitaires de Chargement et Validation

In [ ]:
def load_puzzles(filepath: str, max_puzzles: int = None) -> List[str]:
    """Charge les puzzles depuis un fichier."""
    puzzles = []
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if len(line) >= 81:
                puzzles.append(line[:81])
                if max_puzzles and len(puzzles) >= max_puzzles:
                    break
    return puzzles

def puzzle_to_grid(puzzle_str: str) -> List[List[int]]:
    """Convertit une chaine de 81 caracteres en grille 9x9."""
    return [[int(puzzle_str[i * 9 + j]) if puzzle_str[i * 9 + j] in '123456789' else 0 
             for j in range(9)] for i in range(9)]

def grid_to_jax(grid: List[List[int]]) -> jnp.ndarray:
    """Convertit une grille 2D en tableau JAX 1D."""
    return jnp.array([cell for row in grid for cell in row])

def jax_to_grid(flat: jnp.ndarray) -> List[List[int]]:
    """Convertit un tableau JAX 1D en grille 2D."""
    flat = np.array(flat)
    return [[int(flat[i * 9 + j]) for j in range(9)] for i in range(9)]

def verify_solution(grid: List[List[int]]) -> bool:
    """Verifie qu'une solution Sudoku est valide."""
    for row in grid:
        if sorted(row) != list(range(1, 10)):
            return False
    for c in range(9):
        col = [grid[r][c] for r in range(9)]
        if sorted(col) != list(range(1, 10)):
            return False
    for br in range(3):
        for bc in range(3):
            box = [grid[br*3+r][bc*3+c] for r in range(3) for c in range(3)]
            if sorted(box) != list(range(1, 10)):
                return False
    return True

def count_errors(grid: List[List[int]]) -> int:
    """Compte le nombre d'erreurs dans une grille."""
    errors = 0
    for row in grid:
        seen = set()
        for cell in row:
            if cell in seen:
                errors += 1
            elif cell > 0:
                seen.add(cell)
    for c in range(9):
        seen = set()
        for r in range(9):
            cell = grid[r][c]
            if cell in seen:
                errors += 1
            elif cell > 0:
                seen.add(cell)
    for br in range(3):
        for bc in range(3):
            seen = set()
            for r in range(3):
                for c in range(3):
                    cell = grid[br*3+r][bc*3+c]
                    if cell in seen:
                        errors += 1
                    elif cell > 0:
                        seen.add(cell)
    return errors

def print_grid(grid: List[List[int]], title: str = ""):
    """Affiche une grille Sudoku."""
    if title:
        print(f"\n{title}")
    print("-" * 25)
    for i, row in enumerate(grid):
        if i % 3 == 0 and i > 0:
            print("|" + "-" * 23 + "|")
        line = "| "
        for j, cell in enumerate(row):
            if j % 3 == 0 and j > 0:
                line += "| "
            line += f"{cell if cell > 0 else ' '} "
        line += "|"
        print(line)
    print("-" * 25)

# Chargement des puzzles
possible_paths = [
    "Puzzles/Sudoku_Easy51.txt",
    "MyIA.AI.Notebooks/Sudoku/Puzzles/Sudoku_Easy51.txt",
]

puzzles = []
for path in possible_paths:
    try:
        puzzles = load_puzzles(path, max_puzzles=5)
        if puzzles:
            print(f"{len(puzzles)} puzzles charges depuis {path}")
            break
    except FileNotFoundError:
        continue

if not puzzles:
    print("Utilisation d'un puzzle de test par defaut")
    test_puzzle_str = "900200543100063025508407060026309001057010290090670530240530600705200304080041950"
    puzzles = [test_puzzle_str]

## 3. Modele NumPyro avec Dirichlet et Contraintes Douces

L'approche utilise des distributions **Dirichlet** pour modeliser les probabilites des valeurs de chaque cellule. Les contraintes de Sudoku sont encodees comme des **contraintes douces** via des penalites dans la fonction de perte.

### Principe du modele

1. **Variables latentes** : Parametres de concentration Dirichlet `alpha` pour chaque cellule
2. **Observations** : Les cellules connues fixent leur distribution
3. **Contraintes** : Penalites pour les conflits lignes/colonnes/blocs

In [ ]:
def compute_soft_constraint_penalty(cell_probs: jnp.ndarray) -> jnp.ndarray:
    """
    Calcule une penalite douce pour les violations de contraintes Sudoku.
    
    Au lieu d'utiliser des operations discretes (==), on utilise des produits
    de probabilites pour detecter les conflits de maniere differentiable.
    
    Args:
        cell_probs: Probabilites des cellules, shape (81, 9)
    
    Returns:
        Penalite totale (scalaire)
    """
    penalty = 0.0
    
    # Penalites pour les lignes
    for r in range(9):
        row_probs = cell_probs[r * 9:(r + 1) * 9]  # (9, 9)
        # Pour chaque valeur, la probabilite qu'elle apparaisse plus d'une fois
        # est approximee par la somme des produits de paires
        for v in range(9):
            value_probs = row_probs[:, v]  # (9,)
            # Somme des produits p_i * p_j pour i != j (conflit potentiel)
            sum_probs = jnp.sum(value_probs)
            sum_sq = jnp.sum(value_probs ** 2)
            conflict_score = (sum_probs ** 2 - sum_sq) / 2
            penalty = penalty + conflict_score
    
    # Penalites pour les colonnes
    for c in range(9):
        col_probs = cell_probs[c::9]  # (9, 9)
        for v in range(9):
            value_probs = col_probs[:, v]
            sum_probs = jnp.sum(value_probs)
            sum_sq = jnp.sum(value_probs ** 2)
            conflict_score = (sum_probs ** 2 - sum_sq) / 2
            penalty = penalty + conflict_score
    
    # Penalites pour les blocs 3x3
    for br in range(3):
        for bc in range(3):
            indices = []
            for i in range(3):
                for j in range(3):
                    indices.append((br * 3 + i) * 9 + (bc * 3 + j))
            box_probs = cell_probs[jnp.array(indices)]  # (9, 9)
            for v in range(9):
                value_probs = box_probs[:, v]
                sum_probs = jnp.sum(value_probs)
                sum_sq = jnp.sum(value_probs ** 2)
                conflict_score = (sum_probs ** 2 - sum_sq) / 2
                penalty = penalty + conflict_score
    
    return penalty


def dirichlet_sudoku_model(initial_grid: jnp.ndarray, constraint_weight: float = 10.0):
    """
    Modele NumPyro pour Sudoku avec distributions Dirichlet.

    Args:
        initial_grid: Grille initiale (81,), valeurs 0-9 (0 = vide)
        constraint_weight: Poids des contraintes dans la fonction de perte
    """
    n_cells = 81
    n_values = 9

    # Parametres de concentration Dirichlet (variables latentes)
    # On utilise un prior Dirichlet uniforme comme base
    alpha_base = numpyro.param(
        "alpha_base",
        jnp.ones((n_cells, n_values)),
        constraint=dist.constraints.positive
    )

    # Ajuster les alpha pour les cellules connues (version vectorisee JAX)
    epsilon = 1e-3
    fixed_value = 1000.0  # Valeur elevee pour fixer une cellule

    # Creer un mask pour les cellules connues (version vectorisee)
    is_known = (initial_grid > 0)[:, None]  # (81, 1)
    
    # One-hot encoding des valeurs connues (FIXE: version compatible JIT)
    # Pour les cellules vides (valeur 0), on cree un one-hot de zeros
    # Pour les cellules connues (1-9), on convertit en indices 0-8
    # IMPORTANT: On ne peut pas passer -1 a one_hot, donc on utilise une approche differente
    
    # Methode: Creer les one-hots pour toutes les valeurs possibles (1-9)
    # puis selectionner celle correspondante ou zeros si vide
    
    # Indices 0-8 pour les valeurs 1-9
    value_indices = jnp.arange(n_values)  # (9,)
    
    # Pour chaque cellule, calculer son one-hot en comparant avec toutes les valeurs
    # shape: (81, 1) x (9,) -> (81, 9)
    grid_expanded = initial_grid[:, None]  # (81, 1)
    values_expanded = value_indices[None, :] + 1  # (1, 9)
    
    # one_hot est True si grid == value+1, False sinon
    known_values_onehot = (grid_expanded == values_expanded).astype(jnp.float32)
    
    # Masquer les cellules vides (valeur 0)
    known_values_onehot = known_values_onehot * is_known.astype(jnp.float32)

    # Alpha pour cellules connues: haute concentration sur la valeur connue
    alpha_known = known_values_onehot * fixed_value + (1 - known_values_onehot) * epsilon

    # Alpha pour cellules inconnues: utiliser les parametres appris
    alpha_unknown = alpha_base

    # Combiner en utilisant where (vectorise, compatible JIT)
    alpha = jnp.where(is_known.astype(jnp.bool_), alpha_known, alpha_unknown)

    # Echantillonner les probabilites des cellules depuis Dirichlet
    # Note: On utilise un echantillonnage reparametrise pour SVI
    with numpyro.plate("cells", n_cells):
        cell_probs = numpyro.sample(
            "cell_probs",
            dist.Dirichlet(alpha)
        )

    # Calculer les penalites de contraintes
    penalty = compute_soft_constraint_penalty(cell_probs)

    # Observer une penalite faible (contrainte douce)
    numpyro.factor("constraint_penalty", -constraint_weight * penalty)

    return cell_probs

### Guide (distribution variationnelle) pour SVI

In [ ]:
def dirichlet_sudoku_guide(initial_grid: jnp.ndarray, constraint_weight: float = 10.0):
    """
    Guide variationnel pour le modele Sudoku.

    Le guide apprend les parametres de concentration Dirichlet pour
    les cellules inconnues.
    """
    n_cells = 81
    n_values = 9

    # Parametres variationnels (concentrations Dirichlet)
    alpha_var = numpyro.param(
        "alpha_var",
        jnp.ones((n_cells, n_values)),
        constraint=dist.constraints.positive
    )

    # Fixer les cellules connues (version vectorisee JAX - FIXE)
    epsilon = 1e-3
    fixed_value = 1000.0

    # Creer un mask pour les cellules connues (version vectorisee)
    is_known = (initial_grid > 0)[:, None]  # (81, 1)
    
    # One-hot encoding des valeurs connues (FIXE: version compatible JIT)
    # Meme approche que le modele: comparaison vectorielle au lieu de one_hot
    value_indices = jnp.arange(n_values)  # (9,)
    grid_expanded = initial_grid[:, None]  # (81, 1)
    values_expanded = value_indices[None, :] + 1  # (1, 9)
    
    # one_hot est True si grid == value+1, False sinon
    known_values_onehot = (grid_expanded == values_expanded).astype(jnp.float32)
    
    # Masquer les cellules vides (valeur 0)
    known_values_onehot = known_values_onehot * is_known.astype(jnp.float32)

    # Alpha pour cellules connues
    alpha_known = known_values_onehot * fixed_value + (1 - known_values_onehot) * epsilon

    # Combiner en utilisant where (vectorise, compatible JIT)
    alpha = jnp.where(is_known.astype(jnp.bool_), alpha_known, alpha_var)

    # Echantillonner depuis le guide
    with numpyro.plate("cells", n_cells):
        numpyro.sample("cell_probs", dist.Dirichlet(alpha))

## 4. Solveur Iteratif avec NumPyro

Le solveur combine l'inference probabiliste (SVI) avec une fixation iterative des cellules les plus certaines, inspire du `IterativeSudokuModel` du notebook C#.

In [ ]:
class PyroSudokuSolver:
    """
    Solveur Sudoku utilisant l'inference probabiliste avec NumPyro.
    
    Combine SVI pour l'inference avec une fixation iterative des cellules.
    """
    
    def __init__(self, n_svi_iterations: int = 500, 
                 constraint_weight: float = 10.0,
                 learning_rate: float = 0.05,
                 confidence_threshold: float = 0.5):
        self.n_svi_iterations = n_svi_iterations
        self.constraint_weight = constraint_weight
        self.learning_rate = learning_rate
        self.confidence_threshold = confidence_threshold
    
    def infer_probabilities(self, grid: List[List[int]]) -> np.ndarray:
        """
        Utilise SVI pour inferer les probabilites des cellules.
        
        Returns:
            Tableau (81, 9) des probabilites pour chaque cellule
        """
        initial_flat = grid_to_jax(grid)
        
        # Initialiser le moteur SVI
        rng_key = random.PRNGKey(42)
        optimizer = Adam(step_size=self.learning_rate)
        
        svi = SVI(
            dirichlet_sudoku_model,
            dirichlet_sudoku_guide,
            optimizer,
            loss=Trace_ELBO()
        )
        
        # Executer SVI
        try:
            svi_result = svi.run(
                rng_key, 
                self.n_svi_iterations, 
                initial_flat,
                self.constraint_weight,
                progress_bar=False
            )
            
            # Extraire les parametres appris
            params = svi_result.params
            alpha_var = params["alpha_var"]
            
            # Normaliser pour obtenir les probabilites
            probs = alpha_var / jnp.sum(alpha_var, axis=-1, keepdims=True)
            
            return np.array(probs)
            
        except Exception as e:
            print(f"Erreur SVI: {e}")
            # Retourner des probabilites uniformes en cas d'erreur
            return np.ones((81, 9)) / 9
    
    def solve(self, grid: List[List[int]], max_iterations: int = 100) -> Tuple[List[List[int]], Dict]:
        """
        Resout un Sudoku en fixant iterativement les cellules les plus certaines.
        
        Args:
            grid: Grille Sudoku 9x9
            max_iterations: Nombre max d'iterations
        
        Returns:
            Tuple (solution, metadata)
        """
        grid = [row[:] for row in grid]  # Copie
        
        metadata = {
            'iterations': 0,
            'cells_fixed': [],
            'confidences': [],
            'converged': False,
            'errors_at_end': 0,
            'svi_calls': 0
        }
        
        for iteration in range(max_iterations):
            metadata['iterations'] = iteration + 1
            
            # Inferer les probabilites
            probs = self.infer_probabilities(grid)
            metadata['svi_calls'] += 1
            
            # Trouver la meilleure cellule a fixer
            best_cell = None
            best_confidence = 0
            
            for i in range(9):
                for j in range(9):
                    if grid[i][j] == 0:
                        idx = i * 9 + j
                        confidence = float(jnp.max(probs[idx]))
                        
                        if confidence > best_confidence:
                            best_confidence = confidence
                            best_value = int(jnp.argmax(probs[idx])) + 1
                            best_cell = (i, j, best_value, confidence)
            
            # Verifier si on peut fixer une cellule
            if best_cell is None or best_confidence < self.confidence_threshold:
                break
            
            # Fixer la cellule
            i, j, value, confidence = best_cell
            grid[i][j] = value
            metadata['cells_fixed'].append((i, j, value, confidence))
            metadata['confidences'].append(confidence)
        
        metadata['converged'] = verify_solution(grid)
        metadata['errors_at_end'] = count_errors(grid)
        
        return grid, metadata

### Test du solveur NumPyro

In [ ]:
# Test sur le premier puzzle
test_grid = puzzle_to_grid(puzzles[0])
print_grid(test_grid, "Puzzle initial:")

# NOTE: Nombre d'iterations reduit pour demonstration
# En pratique, plus d'iterations = meilleures probabilites mais plus lent
solver = PyroSudokuSolver(
    n_svi_iterations=50,  # Reduit pour execution rapide (CPU)
    constraint_weight=15.0,
    learning_rate=0.05,
    confidence_threshold=0.35  # Seuil plus bas pour compenser moins d'iterations
)

start_time = time.time()
solution, metadata = solver.solve(test_grid)
elapsed = time.time() - start_time

print_grid(solution, f"\nSolution (en {elapsed:.1f}s):")
print(f"\nMetadonnees:")
print(f"  - Iterations: {metadata['iterations']}")
print(f"  - Appels SVI: {metadata['svi_calls']}")
print(f"  - Cellules fixees: {len(metadata['cells_fixed'])}")
print(f"  - Converge: {metadata['converged']}")
print(f"  - Erreurs: {metadata['errors_at_end']}")
if metadata['confidences']:
    print(f"  - Confiance moyenne: {np.mean(metadata['confidences']):.3f}")

## 5. Solveur Hybride : NumPyro + Propagation de Contraintes

Pour ameliorer la robustesse, nous combinons l'inference probabiliste avec une propagation de contraintes deterministe (naked singles).

In [ ]:
class HybridPyroSolver:
    """
    Solveur hybride combinant propagation de contraintes deterministe
    et inference probabiliste NumPyro.
    
    La propagation deterministe resout les cellules "faciles" (naked singles),
    laissant l'inference probabiliste gerer les cas ambigus.
    """
    
    def __init__(self, n_svi_iterations: int = 200,
                 constraint_weight: float = 10.0,
                 learning_rate: float = 0.05,
                 confidence_threshold: float = 0.3):
        self.pyro_solver = PyroSudokuSolver(
            n_svi_iterations=n_svi_iterations,
            constraint_weight=constraint_weight,
            learning_rate=learning_rate,
            confidence_threshold=confidence_threshold
        )
    
    def solve(self, grid: List[List[int]], max_iterations: int = 100) -> Tuple[List[List[int]], Dict]:
        """Resout un Sudoku en alternant propagation et inference."""
        grid = [row[:] for row in grid]
        
        metadata = {
            'deterministic_fixes': 0,
            'probabilistic_fixes': 0,
            'iterations': 0,
            'converged': False,
            'errors_at_end': 0
        }
        
        for iteration in range(max_iterations):
            metadata['iterations'] = iteration + 1
            
            # Etape 1: Propagation deterministe (naked singles)
            fixed_det = self._deterministic_propagation(grid)
            metadata['deterministic_fixes'] += fixed_det
            
            # Verifier si resolu
            if self._is_solved(grid):
                metadata['converged'] = True
                break
            
            # Si la propagation a progresse, continuer avec elle
            if fixed_det > 0:
                continue
            
            # Etape 2: Inference probabiliste pour debloquer
            probs = self.pyro_solver.infer_probabilities(grid)
            
            # Trouver la meilleure cellule parmi celles avec plusieurs candidats
            best = self._get_best_probabilistic_cell(probs, grid)
            
            if best:
                i, j, value, confidence = best
                grid[i][j] = value
                metadata['probabilistic_fixes'] += 1
            else:
                break  # Bloque
        
        metadata['converged'] = verify_solution(grid)
        metadata['errors_at_end'] = count_errors(grid)
        
        return grid, metadata
    
    def _deterministic_propagation(self, grid: List[List[int]]) -> int:
        """Applique la propagation de contraintes (naked singles)."""
        fixed = 0
        changed = True
        
        while changed:
            changed = False
            for i in range(9):
                for j in range(9):
                    if grid[i][j] == 0:
                        candidates = self._get_candidates(grid, i, j)
                        if len(candidates) == 1:
                            grid[i][j] = candidates[0]
                            fixed += 1
                            changed = True
        
        return fixed
    
    def _get_candidates(self, grid: List[List[int]], row: int, col: int) -> List[int]:
        """Retourne les candidats possibles pour une cellule."""
        used = set()
        used.update(grid[row])
        used.update(grid[r][col] for r in range(9))
        br, bc = (row // 3) * 3, (col // 3) * 3
        for r in range(br, br + 3):
            for c in range(bc, bc + 3):
                used.add(grid[r][c])
        return [v for v in range(1, 10) if v not in used]
    
    def _get_best_probabilistic_cell(self, probs: np.ndarray, grid: List[List[int]]) -> Optional[Tuple]:
        """Trouve la meilleure cellule probabiliste parmi les candidats valides."""
        best = None
        best_confidence = 0
        
        for i in range(9):
            for j in range(9):
                if grid[i][j] == 0:
                    candidates = self._get_candidates(grid, i, j)
                    if len(candidates) > 1:
                        idx = i * 9 + j
                        for c in candidates:
                            prob = probs[idx, c - 1]
                            if prob > best_confidence:
                                best_confidence = prob
                                best = (i, j, c, prob)
        
        if best_confidence >= self.pyro_solver.confidence_threshold:
            return best
        return None
    
    def _is_solved(self, grid: List[List[int]]) -> bool:
        """Verifie si la grille est complete."""
        return all(cell > 0 for row in grid for cell in row)

### Test du solveur hybride

In [ ]:
print("=== Test du Solveur Hybride NumPyro ===")
print("NOTE: Test sur 2 puzzles pour demonstration\n")

hybrid_solver = HybridPyroSolver(
    n_svi_iterations=50,  # Reduit pour demo
    constraint_weight=10.0,
    confidence_threshold=0.3
)

for i, puzzle_str in enumerate(puzzles[:2]):
    test_grid = puzzle_to_grid(puzzle_str)
    
    start = time.time()
    solution, meta = hybrid_solver.solve(test_grid)
    elapsed = time.time() - start
    
    valid = verify_solution(solution)
    status = "OK" if valid else f"INVALIDE ({meta['errors_at_end']} erreurs)"
    print(f"Puzzle {i+1}: {status} ({elapsed:.1f}s)")
    print(f"  - Fixes deterministes: {meta['deterministic_fixes']}")
    print(f"  - Fixes probabilistes: {meta['probabilistic_fixes']}")
    print(f"  - Iterations: {meta['iterations']}")

### Interpretation des resultats

Le solveur hybride combine les forces des deux approches :

| Technique | Role | Avantage |
|-----------|------|----------|
| **Propagation deterministe** | Resolution des naked singles | Tres rapide, garantie locale |
| **Inference NumPyro** | Deblocage des cas ambigus | Explore les alternatives |

**Observation** : La propagation deterministe resout generalement la majorite des cellules, l'inference probabiliste n'etant necessaire que pour quelques cellules difficiles.

## 6. Benchmark Comparatif

In [ ]:
def benchmark_solvers(puzzles: List[str], limit: int = 2):
    """Compare les differentes approches."""
    # NOTE: Iterations reduites pour demonstration (CPU)
    # L'inference probabiliste est beaucoup plus lente que la propagation deterministe
    solvers = {
        "Hybride NumPyro": HybridPyroSolver(30, 10.0, 0.05, 0.3),
    }
    
    results = {name: {"solved": 0, "total_time": 0, "errors": 0} for name in solvers}
    
    for i, puzzle_str in enumerate(puzzles[:limit]):
        grid = puzzle_to_grid(puzzle_str)
        print(f"\n--- Puzzle {i+1} ---")
        
        for name, solver in solvers.items():
            test_grid = [row[:] for row in grid]
            
            start = time.time()
            solution, meta = solver.solve(test_grid)
            elapsed = time.time() - start
            
            if verify_solution(solution):
                results[name]["solved"] += 1
                results[name]["total_time"] += elapsed
                status = "OK"
            else:
                errors = count_errors(solution)
                results[name]["errors"] += errors
                status = f"INVALIDE ({errors} erreurs)"
            
            print(f"  {name}: {status} ({elapsed:.1f}s)")
            print(f"    Fixes deterministes: {meta['deterministic_fixes']}")
            print(f"    Fixes probabilistes: {meta['probabilistic_fixes']}")
    
    # Resume
    print("\n" + "=" * 55)
    print("RESUME")
    print("=" * 55)
    print(f"{'Solveur':<25} {'Resolus':<10} {'Temps moy':<12} {'Erreurs':<10}")
    print("-" * 55)
    
    for name, data in results.items():
        solved = data["solved"]
        avg_time = data["total_time"] / max(solved, 1)
        print(f"{name:<25} {solved}/{limit:<8} {avg_time:.1f}s{' '*5} {data['errors']}")
    
    print("\nNOTE: Les solveurs probabilistes sont plus lents car ils")
    print("necessitent plusieurs iterations d'inference pour apprendre les")
    print("distributions. Pour des puzzles plus difficiles, augmentez")
    print("n_svi_iterations (300+ recommande pour production).")
    
    return results

# Executer le benchmark (reduit pour la demo)
print("=== Benchmark NumPyro sur puzzles Easy ===")
print("NOTE: Benchmark reduit pour demonstration (2 puzzles, 30 iterations)")
print("En production, utilisez plus d'iterations et plus de puzzles.\n")
results = benchmark_solvers(puzzles, limit=2)

## 7. Comparaison avec Infer.NET (C#)

In [ ]:
# Tableau comparatif
comparison_data = {
    "Aspect": [
        "Bibliotheque",
        "Algorithme d'inference",
        "Contraintes",
        "Variables discretes",
        "Modele de base",
        "Solveur iteratif",
        "Precompilation",
        "Performance (Easy)",
        "Performance (Medium)",
        "Garantie de solution"
    ],
    "Infer.NET (C#)": [
        "Microsoft.ML.Probabilistic",
        "Expectation Propagation",
        "Dures (ConstrainFalse)",
        "Natives",
        "NaiveProbabilisticSolver",
        "IterativeSudokuModel",
        "Oui (DLL generee)",
        "~1-5s",
        "Variable",
        "Partielle"
    ],
    "NumPyro (Python)": [
        "NumPyro + JAX",
        "SVI (Variational Inference)",
        "Douces (numpyro.factor)",
        "Via Dirichlet (continu)",
        "dirichlet_sudoku_model",
        "PyroSudokuSolver",
        "JIT compilation",
        "~5-15s",
        "Variable",
        "Partielle"
    ]
}

import pandas as pd
df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

### Lecons retenues

1. **Infer.NET reste superieur pour les CSP** : Expectation Propagation gere nativement les contraintes discretes

2. **NumPyro fonctionne avec des astuces** : L'utilisation de Dirichlet + contraintes douces permet de contourner les limitations des variables discretes

3. **L'approche hybride est recommandee** : Combiner propagation deterministe et inference probabiliste donne les meilleurs resultats

4. **Pour la production** : Utiliser OR-Tools, Z3 ou Choco qui sont concus pour les CSP

## 8. Conclusion

### Ce que nous avons appris

| Concept | Application NumPyro |
|---------|---------------------|
| **Distributions Dirichlet** | Modeliser les probabilites des cellules |
| **Contraintes douces** | `numpyro.factor` pour penaliser les conflits |
| **SVI** | Inference variationnelle pour apprendre les distributions |
| **Resolution iterative** | Fixer les cellules les plus certaines |

### Pourquoi cette approche reste educative

La programmation probabiliste avec NumPyro/Pyro est excellente pour :
- L'apprentissage bayesien
- Les modeles hierarchiques
- L'incertitude epistemique

Mais elle **n'est pas optimale** pour :
- La satisfaction de contraintes pures
- L'optimisation combinatoire

> **Recommandation** : Pour resoudre des Sudokus en production, utiliser OR-Tools, Z3 ou Choco. La programmation probabiliste est un outil pedagogique pour comprendre l'inference bayesienne.

---

**Navigation** : [<< Choco](Sudoku-11-Choco-Python.ipynb) | [Index](README.md) | [Neural Network >>](Sudoku-16-NeuralNetwork-Python.ipynb)

**Voir aussi** :
- [Sudoku-15-Infer-Csharp](Sudoku-15-Infer-Csharp.ipynb) - Version Infer.NET (C#)
- [Probas](../Probas/README.md) - Serie complete sur la programmation probabiliste
- [Sudoku-10-ORTools-Python](Sudoku-10-ORTools-Python.ipynb) - Approche CP-SAT (recommandee)